In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

## **Parte 1**

In [ ]:
#Para subir el archivo
from google.colab import files
uploaded = files.upload()  

Saving Frogs_MFCCs.csv to Frogs_MFCCs.csv


In [ ]:
!ls

Frogs_MFCCs.csv  sample_data


In [ ]:
df = pd.read_csv('Frogs_MFCCs.csv')
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,MFCCs_11,MFCCs_12,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,0.188654,-0.075622,-0.156436,0.082245,0.135752,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,0.270958,-0.095004,-0.254341,0.022786,0.163320,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,0.266064,-0.072827,-0.237384,0.050791,0.207338,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,0.267279,-0.162258,-0.317084,-0.011567,0.100413,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,0.332695,-0.100749,-0.298524,0.037439,0.219153,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,0.065905,-0.000717,-0.145668,-0.059364,0.024206,-0.000861,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,0.038545,0.000856,-0.164675,-0.105600,0.030767,0.006457,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,0.062748,0.011868,-0.150025,-0.078615,0.024861,0.008696,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,0.049792,-0.003997,-0.153120,-0.075320,0.022903,0.001924,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [ ]:
#Separamos en caracteristicas y clase
MFCCs = df.iloc[:, :22]
Species = df['Species']
tipos = []
#Hacemos que las clases vayan de 0 a 9
for i in range(len(Species)):
  if Species[i] not in tipos:
    tipos.append(Species[i])
  Species[i] = tipos.index(Species[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## **Parte 2**

In [ ]:
def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean", sample_size=300,)
    ]

    clusterizacion = estimator[-1].labels_ #Obtenemos las etiquetas de cada punto
    nro_clusters = len(set(clusterizacion)) #Vemos cuantos clusters hay

    results += [nro_clusters] #ponemos la cantidad de clusters obtenidos a los resultados
    # Show the results
    formatter_result = ("{:9s}\t{:.3f}\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t\t{:0}")
    print(formatter_result.format(*results))


## **Parte 3**

In [ ]:
def bench_clustering2(kmeans, name, data, labels, bool): 
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    bool : bool
        Indica si se utilizan todas las muestras (True) o no (False)
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, 'NAN']

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]

    #Estos parametros son usados si bool es True y si es False:
    clusterizacion = estimator[-1].labels_ #estimacion inicial
    new_clustering = [] #Aca guardamos los labels de los datos finales, los labels obtenidos
                        #dependeran de si bool es True o False
    #---------------------------------------------------------------------------
    #Estos otros parametros son usados solo si bool es False, para el caso 
    new_labels = [] #Aca se guardaran los labels correctos
    new_data = pd.DataFrame() #Aca se guardaran los datos que obtuvieron un label distinto a -1
    #---------------------------------------------------------------------------
    #Si bool es true significa que usare todos los datos, incluyendo aquellos 
    #datos que no tienen un cluster asociado (y que por tanto estan en el
    #cluster -1)
    if bool == True:  
      maximo = max(clusterizacion)
      new_labels = labels
      new_data = data 
      for i in range(len(clusterizacion)):
        if clusterizacion[i] == -1: #Los datos no asociados a un cluster se consideran
                            # en un cluster propio
          new_clustering.append(maximo+1)
        else:
          new_clustering.append(clusterizacion[i])

    #Si bool es false significa que solo considerare los datos que tienen 
    #un cluster asociado distinto de -1
    elif bool == False:
      for i in range(len(clusterizacion)):
        if clusterizacion[i] != -1: #si el elemento si esta asociado a un cluster
          new_clustering.append(clusterizacion[i]) #Guardamos la clusterizacion del elemento
          new_labels.append(labels[i]) #Guardamos la correcta clasificacion del elemento
          new_data = pd.concat([new_data, data.iloc[i]], axis=1)
        #Los elemenotos con cluster -1 no se consideran
      new_data = new_data.transpose() #Para que las caracteristicas queden como columnas

    #Cantidad de clusters
    nro_clusters = len(set(new_clustering))
    
    try:
      if nro_clusters <= 1: #Solo hay 1 cluster o no hay ninguno
        #Obtenemos las metricas
        results += [m(new_labels, new_clustering) for m in clustering_metrics]
        results +=['NAN'] #No hay silhuette
        results += [nro_clusters] #ponemos la cantidad de clusters obtenidos a los resultados
        # Show the results
        formatter_result = ("{:9s}\t{:.3f}\t{:3s}\t{:.3f}\t{:.3f}"
                         "\t{:.3f}\t{:.3f}\t{:.3f}\t{:3s}\t\t{:0}")
        print(formatter_result.format(*results))

      elif nro_clusters > 1: #Hay mas de 1 cluster
        #Obtenemos las metricas
        results += [m(new_labels, new_clustering) for m in clustering_metrics]
        #The silhouette score requires the full dataset
        results += [
           metrics.silhouette_score(data, estimator[-1].labels_,
                                  metric="euclidean", sample_size=300,)
        ]
        results += [nro_clusters] #ponemos la cantidad de clusters obtenidos a los resultados
        # Show the results
        formatter_result = ("{:9s}\t{:.3f}\t{:3s}\t{:.3f}\t{:.3f}"
                           "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t\t{:0}")
        print(formatter_result.format(*results))

    except:
      print("Para el caso "+str(name)+" hubo un error al calcular las metricas")


## **Parte 4**

In [ ]:
#Pruebas:

print(98 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette\tn_clusters')
n_clusters_pred = 10 #Cantidad de clusters a usar en KMeans y Aglomerative
#Kmeans con inicializacion al azar
kmeans = KMeans(init="random", n_clusters=n_clusters_pred, n_init=4,
                random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=MFCCs, labels=Species)

#Kmeans++
kmeans = KMeans(init="k-means++", n_clusters=n_clusters_pred, n_init=4,
                random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=MFCCs, labels=Species)

#DBSCAN con epsilon por defecto (0.5)
dbscan = DBSCAN(eps=0.5).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.5-f', data=MFCCs, labels=Species, bool=False)

#DBSCAN con epsilon 0.7
dbscan = DBSCAN(eps=0.7).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.7-f', data=MFCCs, labels=Species, bool=False)

#DBSCAN con epsilon 0.2
dbscan = DBSCAN(eps=0.2).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.2-f', data=MFCCs, labels=Species, bool=False)

#DBSCAN con epsilon por defecto (0.5), agregando outliers a cluster extra
dbscan = DBSCAN(eps=0.5).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.5-t', data=MFCCs, labels=Species, bool=True)

#DBSCAN con epsilon 0.7, agregando outliers a cluster extra
dbscan = DBSCAN(eps=0.7).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.7-t', data=MFCCs, labels=Species, bool=True)

#DBSCAN con epsilon 0.2, agregando outliers a cluster extra
dbscan = DBSCAN(eps=0.2).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.2-t', data=MFCCs, labels=Species, bool=True)

#Clustering aglomerativo
aglomerative = AgglomerativeClustering(n_clusters=n_clusters_pred)
bench_clustering2(kmeans=aglomerative, name='Aglomerative', data=MFCCs, labels=Species, bool=True)
print(98 * '_')
#-------------------------------------------------------------------------------
#Aplicando PCA
MFCCs_red = PCA(n_components=2).fit_transform(MFCCs)
MFCCs_red_df = pd.DataFrame(MFCCs_red)

print("")
print("Al aplicar PCA se obtienen los siguientes resultados:")
print(98 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette\tn_clusters')
#Kmeans con inicializacion al azar
kmeans = KMeans(init="random", n_clusters=n_clusters_pred, n_init=10)
bench_k_means(kmeans=kmeans, name="random", data=MFCCs_red_df, labels=Species)

#Kmeans++
kmeans = KMeans(init="k-means++", n_clusters=n_clusters_pred, n_init=10)
bench_k_means(kmeans=kmeans, name="k-means++", data=MFCCs_red_df, labels=Species)

#DBSCAN con epsilon por defecto (0.5)
dbscan = DBSCAN(eps=0.5).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.5-f', data=MFCCs_red_df, labels=Species, bool=False)

#DBSCAN con epsilon 0.7
dbscan = DBSCAN(eps=0.7).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.7-f', data=MFCCs_red_df, labels=Species, bool=False)

#DBSCAN con epsilon 0.2
dbscan = DBSCAN(eps=0.2).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.2-f', data=MFCCs_red_df, labels=Species, bool=False)

#DBSCAN con epsilon por defecto (0.5), agregando outliers a cluster extra
dbscan = DBSCAN(eps=0.5).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.5-t', data=MFCCs_red_df, labels=Species, bool=True)

#DBSCAN con epsilon 0.7, agregando outliers a cluster extra
dbscan = DBSCAN(eps=0.7).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.7-t', data=MFCCs_red_df, labels=Species, bool=True)

#DBSCAN con epsilon 0.2, agregando outliers a cluster extra
dbscan = DBSCAN(eps=0.2).fit(MFCCs)
bench_clustering2(kmeans= dbscan, name='DBSCAN-0.2-t', data=MFCCs_red_df, labels=Species, bool=True)

#Clustering aglomerativo
aglomerative = AgglomerativeClustering(n_clusters=n_clusters_pred)
bench_clustering2(kmeans=aglomerative, name='Aglomerative', data=MFCCs_red_df, labels=Species, bool=True)

print(98 * '_')

__________________________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette	n_clusters
random   	0.316	54400	0.641	0.507	0.566	0.326	0.565	0.217		10
k-means++	0.301	51996	0.708	0.571	0.633	0.432	0.631	0.256		10
DBSCAN-0.5-f	0.537	NAN	1.000	0.327	0.493	0.256	0.483	-0.202		8
DBSCAN-0.7-f	0.675	NAN	1.000	0.585	0.738	0.450	0.730	-0.237		13
DBSCAN-0.2-f	0.335	NAN	1.000	0.000	0.000	0.000	0.000	-0.080		2
DBSCAN-0.5-t	0.541	NAN	0.074	0.451	0.127	0.049	0.123	-0.175		9
DBSCAN-0.7-t	0.673	NAN	0.179	0.545	0.270	0.133	0.265	-0.235		14
DBSCAN-0.2-t	0.323	NAN	0.011	0.677	0.022	0.006	0.021	-0.132		3
Aglomerative	2.223	NAN	0.789	0.661	0.719	0.590	0.718	0.239		10
__________________________________________________________________________________________________

Al aplicar PCA se obtienen los siguientes resultados:
__________________________________________________________________________________________________
init		time	ine